In [3]:
import pandas as pd
import librosa
import os
import validators

In [2]:
metadata = pd.read_csv('OADTM2205-raw.csv')
#remove skill_name is NaN and skill_name is olli.others
metadata = metadata[~metadata['skill_name'].isna()]
metadata = metadata[metadata['skill_name'] != 'olli.others']
metadata.to_csv('metadata/raw_multiskill.csv', index=False)

In [15]:
# metadata2 = pd.read_csv('OADTM2205-raw.csv')
# #remove all skill_name, just keep both NaN and olli.others
# metadata3 = metadata2[metadata2['skill_name'].isna()].append(metadata2[metadata2['skill_name'] == 'olli.others'])
# metadata3.to_csv('metadata/raw_others.csv', index=False)


C:\Users\admin\AppData\Local\Temp\ipykernel_10768\2751833123.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metadata3 = metadata2[metadata2['skill_name'].isna()].append(metadata2[metadata2['skill_name'] == 'olli.others'])


In [82]:
csv2 = pd.read_csv('metadata/raw_multiskill.csv')

In [86]:
#export skill_name, percent of skill_name
csv2_x = csv2.groupby('skill_name').size().reset_index(name='count')
csv2_x['percent'] = csv2_x['count']/csv2_x['count'].sum()*100
csv2_x

,skill_name,count,percent
0,Horoscope,271,0.051724
1,Riddles,852,0.162616
2,Translator,1166,0.222548
3,iot,24936,4.759387
4,olli.calculator,1045,0.199453
5,olli.playmusic,118709,22.657286
6,olli.reminder,4358,0.831786
7,olli.weather,9992,1.907114
8,public,362599,69.207131
9,Đặt vé máy bay,5,0.000954


In [94]:
#statistics of skill_name, user_id, percent of skill_name with user_id
csv2_y = csv2.groupby(['skill_name', 'user_id']).size().reset_index(name='number_of_utterances')
#csv2_y = csv2.groupby('skill_name').user_id.nunique().reset_index(name='number_of_user')
#merge csv2_x and csv2_y by skill_name
csv2_z = csv2_x.merge(csv2_y, on='skill_name')
csv2_z['percent(user | skill)'] = csv2_z['number_of_utterances']/csv2_z['count']*100
#delete percen
csv2_z.drop(['percent'], axis=1, inplace=True)
print(csv2_z)
csv2_z.to_csv('metadata/user_by_skill_stat.csv', index=False)


#number of user_id with skill_name
# csv2_y_x = csv2_y.groupby('skill_name').user_id.nunique().reset_index(name='count')
#percent = sum of count of each skill_name/sum of count of all skill_name
# csv2_y['percent'] = csv2_y['count']/csv2_y['count'].sum()*100
# csv2_y.to_csv('metadata/user_by_skill_stat.csv', index=False)

           skill_name   count  user_id  number_of_utterances  \
0           Horoscope     271      108                     6   
1           Horoscope     271      253                    10   
2           Horoscope     271      385                     4   
3           Horoscope     271     6351                    14   
4           Horoscope     271     7141                    31   
...               ...     ...      ...                   ...   
14105          public  362599    23031                    70   
14106          public  362599    23032                     4   
14107          public  362599    23036                     2   
14108          public  362599    23106                     5   
14109  Đặt vé máy bay       5    21792                     5   

       percent(user | skill)  
0                   2.214022  
1                   3.690037  
2                   1.476015  
3                   5.166052  
4                  11.439114  
...                      ...  
14105         

In [96]:
# csv2_10per = csv2.groupby(['skill_name', 'user_id']).apply(lambda x: x.sample(frac=0.1))


csv2_10per = csv2.groupby(['skill_name', 'user_id']).apply(lambda x: x.sample(max(1, len(x)//10)))
csv2_10per.to_csv('metadata/raw_multiskill_10per.csv', index=False)

#export skill_name, percent of skill_name


In [97]:
csv2_10per = pd.read_csv('metadata/raw_multiskill_10per.csv')
csv2_10per_x = csv2_10per.groupby('skill_name').size().reset_index(name='count')
csv2_10per_x['percent'] = csv2_10per_x['count']/csv2_10per_x['count'].sum()*100
csv2_10per_x.to_csv('metadata/skill_stat_10per.csv', index=False)

In [99]:
csv2_10per_y = csv2_10per.groupby(['skill_name', 'user_id']).size().reset_index(name='number_of_utterances')
csv2_10per_z = csv2_10per_x.merge(csv2_10per_y, on='skill_name')
csv2_10per_z['percent(user | skill)'] = csv2_10per_z['number_of_utterances']/csv2_10per_z['count']*100
csv2_10per_z.drop(['percent'], axis=1, inplace=True)
csv2_10per_z.to_csv('metadata/user_by_skill_stat_10per.csv', index=False)
print(csv2_10per_z)

           skill_name  count  user_id  number_of_utterances  \
0           Horoscope     29      108                     1   
1           Horoscope     29      253                     1   
2           Horoscope     29      385                     1   
3           Horoscope     29     6351                     1   
4           Horoscope     29     7141                     3   
...               ...    ...      ...                   ...   
14105          public  35664    23031                     7   
14106          public  35664    23032                     1   
14107          public  35664    23036                     1   
14108          public  35664    23106                     1   
14109  Đặt vé máy bay      1    21792                     1   

       percent(user | skill)  
0                   3.448276  
1                   3.448276  
2                   3.448276  
3                   3.448276  
4                  10.344828  
...                      ...  
14105               0.0196

In [6]:
csv2_10per = pd.read_csv('metadata/raw_multiskill.csv')
#check user_id is NaN
csv2_10per[csv2_10per['user_id'].isna()]
#check utterance_url by validators.url, print False
csv2_10per[csv2_10per['utterance_url'].apply(lambda x: validators.url(x)) == False]

,utterance,utterance_url,intent_name,skill_name,device_id,user_id


In [32]:
import os
import librosa

df = pd.read_csv('metadata/downloaded_10per.csv')
df.columns = ['file_name', 'utterance_url', 'path']

#loop on df
for i in range(10):
    print(df.path.iloc[i])
    with open('metadata/downloaded_10per_final.csv', 'a') as f:
        f.write(df.iloc[i] + '\n')


/content/drive/MyDrive/VAD-Aug/VAD-Aug/downloaded_audio/A1049309953#253#9005547426878363.wav


KeyError: 0